# Assignment 02 - Text Matching System

In [63]:
import torch
import pandas as pd
import numpy as np
import nltk
from nltk.tokenize import TweetTokenizer
from nltk.util import ngrams
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords
import collections
from torchtext.vocab import vocab, GloVe

## 1.  Data Loading

In [64]:
# load data by pandas 'read_csv'
Data = pd.read_csv('data.tsv', sep='\t',error_bad_lines=False)
init_data_len = len(Data)
Data

C:\Users\Tom\AppData\Local\Temp\ipykernel_20596\2577694522.py:2: FutureWarning: The error_bad_lines argument has been deprecated and will be removed in a future version. Use on_bad_lines in the future.


  Data = pd.read_csv('data.tsv', sep='\t',error_bad_lines=False)
b'Skipping line 83032: expected 6 fields, saw 7\n'
b'Skipping line 154657: expected 6 fields, saw 7\n'
b'Skipping line 323916: expected 6 fields, saw 7\n'
C:\Users\Tom\AppData\Local\Temp\ipykernel_20596\2577694522.py:2: DtypeWarning: Columns (1) have mixed types. Specify dtype option on import or set low_memory=False.
  Data = pd.read_csv('data.tsv', sep='\t',error_bad_lines=False)


,id,qid1,qid2,question1,question2,is_duplicate
0,133273,213221,213222.0,How is the life of a math student? Could you d...,Which level of prepration is enough for the ex...,0.0
1,402555,536040,536041.0,How do I control my horny emotions?,How do you control your horniness?,1.0
2,360472,364011,490273.0,What causes stool color to change to yellow?,What can cause stool to come out as little balls?,0.0
3,150662,155721,7256.0,What can one do after MBBS?,What do i do after my MBBS ?,1.0
4,183004,279958,279959.0,Where can I find a power outlet for my laptop ...,"Would a second airport in Sydney, Australia be...",0.0
...,...,...,...,...,...,...
363187,57443,100941,83372.0,How do I make money flying my drone?,How can I use a dji phantom to make money,1.0
363188,278260,62873,34460.0,What can you do with an economics degree?,What jobs can you get with an economics degree?,1.0
363189,136211,217377,217378.0,What type of current does a battery produce?,How does a generator work and produce current?,0.0
363190,302720,425744,285638.0,Grammar: What is difference between schedule a...,How do I understand the difference between the...,0.0


In [65]:
# check if there is any null or NaN in the data set
print(np.sum(np.sum(Data.question1.isnull())))
print(np.sum(np.sum(Data.question2.isnull())))

11
12


In [66]:
# get the index of null data
print('null in question1 are:\n',Data.loc[Data.question1.isnull()].index)
print('null in question2 are:\n',Data.loc[Data.question2.isnull()].index)
q1_null_list = list(Data.loc[Data.question1.isnull()].index)
q2_null_list = list(Data.loc[Data.question2.isnull()].index)
null_list = list(set(q1_null_list + q2_null_list))
print("null_list is:", null_list)

null in question1 are:
 Int64Index([ 36732,  37801,  97828, 130155, 131430, 139005, 228382, 248377,
            300440, 328695, 355331],
           dtype='int64')
null in question2 are:
 Int64Index([ 36732,  37801,  97828, 130155, 131430, 139005, 228382, 300440,
            309620, 328695, 355331, 361568],
           dtype='int64')
null_list is: [361568, 355331, 97828, 131430, 37801, 130155, 309620, 328695, 300440, 248377, 36732, 139005, 228382]


In [67]:
# check if there is any abnormal data (there are some extremely long corrupted data in the dataset)
abnormal_count = 0
for i in Data.index:
    if len(str(Data.question1[i])) > 500 or len(str(Data.question2[i])) > 500:  
        abnormal_count += 1
print('the count of long questions is:', abnormal_count)

the count of long questions is: 39


In [68]:
# in the 39 questions above, 8 of them are corrupted data
# this cell is to remove the null and these abnormal data
bad_line_list = [135940,189732,304107,344961,112298,194953,209649,312959]
Data = Data.drop(null_list, axis=0)
Data = Data.drop(bad_line_list, axis=0)
Data

,id,qid1,qid2,question1,question2,is_duplicate
0,133273,213221,213222.0,How is the life of a math student? Could you d...,Which level of prepration is enough for the ex...,0.0
1,402555,536040,536041.0,How do I control my horny emotions?,How do you control your horniness?,1.0
2,360472,364011,490273.0,What causes stool color to change to yellow?,What can cause stool to come out as little balls?,0.0
3,150662,155721,7256.0,What can one do after MBBS?,What do i do after my MBBS ?,1.0
4,183004,279958,279959.0,Where can I find a power outlet for my laptop ...,"Would a second airport in Sydney, Australia be...",0.0
...,...,...,...,...,...,...
363187,57443,100941,83372.0,How do I make money flying my drone?,How can I use a dji phantom to make money,1.0
363188,278260,62873,34460.0,What can you do with an economics degree?,What jobs can you get with an economics degree?,1.0
363189,136211,217377,217378.0,What type of current does a battery produce?,How does a generator work and produce current?,0.0
363190,302720,425744,285638.0,Grammar: What is difference between schedule a...,How do I understand the difference between the...,0.0


In [69]:
# concatenate question1 and question2 column, and check if there is any duplicated data
q1_q2_concat = pd.concat([Data['question1'], Data['question2']], ignore_index='True')   
dup_ser = q1_q2_concat.duplicated()
print(len(q1_q2_concat))
print(len(dup_ser.loc[dup_ser].index))

726342
233208


In [70]:
# drop the duplicated data
q1_q2_concat.drop_duplicates(inplace=True)
print("length of q1_q2_concat:", len(q1_q2_concat))

length of q1_q2_concat: 493134


In [71]:
# prepare matching data (question2 column)
q2_ser = Data['question2']
q2_dup_ser = q2_ser.duplicated()
print('the length of q2 array is:',len(q2_ser))

the length of q2 array is: 363171


In [72]:
q2_ser.drop_duplicates(inplace = True)
print('the length of q2 array after .duplicated() is:', len(q2_ser))

the length of q2 array after .duplicated() is: 272954


In [73]:
# prepare test data (top 100 with is_duplicate == 1)
Q_100 = Data.loc[Data.is_duplicate==1,['question1','question2']][:100]  
Q_100

,question1,question2
1,How do I control my horny emotions?,How do you control your horniness?
3,What can one do after MBBS?,What do i do after my MBBS ?
7,What is the best self help book you have read?...,What are the top self help books I should read?
11,What will be Hillary Clinton's policy towards ...,What will be Hilary Clinton's policy towards I...
13,Which is the best book to study TENSOR for gen...,Which is the best book for tensor calculus?
...,...,...
251,Is there a directory of landline individual ph...,Is there a directory of individual landline an...
259,Which operating system do Google engineers use?,What operating system do programmers and devel...
262,What is 0 divided by infinity?,What is 0 DIVIDED by 0?
268,Is there proof that alien life exists?,Is there any evidence of alien life in space?


In [74]:
golden_ans_dict = {}
for i in Q_100.index:
    golden_ans_dict[Q_100.question1[i]] = Q_100.question2[i]
print('the length of the dictionary is:',len(golden_ans_dict))

the length of the dictionary is: 100


## 2.  Test Pre-processing

In [75]:
# join all the questions as a long string
q_lst = list(map(str, q1_q2_concat))
q = ' '.join(q_lst).lower()      
print('the length of the string is:',len(q))

the length of the string is: 30955484


In [76]:
# tokenize and lemmatize the string
# this step may take several minutes
tokenizer = TweetTokenizer()
normalizer = WordNetLemmatizer()
q_tokens = tokenizer.tokenize(q)
q_tokens_stm = [normalizer.lemmatize(word) for word in q_tokens]
print('the length of the token list is:',len(q_tokens_stm), '(all the tokens of the string)')

the length of the token list is: 6403921 (all the tokens of the string)


In [77]:
token_set = set()                       # put the lemmatized tokens into a set to get all the unique tokens 
for token in q_tokens_stm:
    if token not in token_set:
        token_set.add(token)
    
token_lst = list(token_set)             # turn the set to a list to get a list of tokens
token_lst.sort()                        # sort the list   
print('the length of the token list is:',len(token_lst))

the length of the token list is: 88504


In [78]:
token_id_dic = {}                       # give all the tokens an id, {t1:0, t2:1, t3:2...}
id_token_dic = {}                       # and make sure that all the tokens are searchable by its id, {0:t1, 1:t2, 2:t3...}
for i in range(len(token_lst)):
    token_id_dic[token_lst[i]] = i
    id_token_dic[i] = token_lst[i]
print('the length of these two dictionaries is:',len(token_id_dic))

the length of these two dictionaries is: 88504


In [79]:
# a pipeline to trun any string to a lemmatized token list
def preprocessing_pipeline(text):       
    text = str(text).lower()
    test_tokens = tokenizer.tokenize(text)
    text_tokens_stm = [normalizer.lemmatize(word) for word in test_tokens]
    return text_tokens_stm

test = "how are you?"
test_tokens = preprocessing_pipeline(test)
print(test_tokens)

['how', 'are', 'you', '?']


## 3. TF-IDF

In [80]:
def unigram_freq(token_list):                 # a function for counting
    unigram_freq = collections.Counter(token_list)
    return unigram_freq

def vectorize(count):                         # use two arrays to store the Counter object seperately
    id_list = []
    count_list = []
    for token in count:
        id_list.append(token_id_dic[token])   # append tokens' id to id list
        count_list.append(count[token])       # append tokens' count to count list
    id_arr = np.array(id_list)                # convet them to array
    count_arr = np.array(count_list)
    return id_arr, count_arr

def text2vec_pipeline(text):                  # a pipeline to trun any string to an id array and a count array
    final_tokens = preprocessing_pipeline(text)
    token_count = unigram_freq(final_tokens)
    id_arr, count_arr = vectorize(token_count)
    return id_arr, count_arr

test_tokens_unigram_freq = unigram_freq(test_tokens)
id_arr, count_arr = vectorize(test_tokens_unigram_freq)
print("unigram frequency of 'how are you?':\n",test_tokens_unigram_freq, '\n')
print("id array and count array of 'how are you?':\n",id_arr, count_arr)

unigram frequency of 'how are you?':
 Counter({'how': 1, 'are': 1, 'you': 1, '?': 1}) 

id array and count array of 'how are you?':
 [38761 10414 87130  6197] [1 1 1 1]


In [81]:
# a function to calculate dft (idf = N/dft, here I follow the formula in the lecture)
def dft(question_list):
    dft_list = []          
    for question in question_list:
        temp_freq = set(preprocessing_pipeline(question))       # a set of all the tokens in a text
        dft_list.extend(temp_freq)                              # extend the list by adding sets of tokens
    dft = unigram_freq(dft_list)                                # calculate the number of occurrences of each token
    return dft

test = ['how are you, how are they','how is this','how is that']
test_dft = dft(test)
dft = dft(q_lst)
print('dft of the test string is:\n', test_dft)

dft of the test string is:
 Counter({'how': 3, 'is': 2, 'they': 1, ',': 1, 'you': 1, 'are': 1, 'this': 1, 'that': 1})


In [82]:
# idf = N/dft. if I have the array of dft, I can compute idf by N/dft. here I follow the equation in the lecture
N = len(q_lst)                  
dft_list = list(dft.items())    # trun the dft to a list to sort it
dft_list.sort()                 # to make sure that dft_list and the token_lst have the same order
dft_dic = dict(dft_list)        # then turn it back to a dictionary to use its keys and values
print("list(dft_dic.keys()) == token_lst:", list(dft_dic.keys()) == token_lst)
dft_arr = np.array(list(dft_dic.values()))
idf_arr = N/dft_arr             # idf = N/dft
print("the shape of the idf array:",idf_arr.shape)      # check if idf_arr includes all the tokens
print("the idf of '?':",idf_arr[6197])                  # 6197 is the id of '?', idf of '?' should be close to 1

list(dft_dic.keys()) == token_lst: True
the shape of the idf array: (88504,)
the idf of '?': 1.001352372133824


In [83]:
inverted_file_dic = {}                      # a dictionary like this: {t1:{d1:0.131, d2:0.254...}, t2:{d5:0.363, d7:0.512...}...}
for i in range(len(token_lst)):             
    inverted_file_dic[token_lst[i]] = {}    # initialize the dictionary
# inverted_file_dic                         # uncomment to check the structure of inverted_file_dic

In [84]:
# as the test cases above, id_arr and count_arr of "how are you?" are [38761 10414 87130  6197] and  [1 1 1 1]
# this function is to add information to the inverted_file_dic
def add_text_to_inv_file_pipeline(text, text_id):      # in q1_q2_concat, each question has an index, so we can retrieve texts by their indexes
    id_arr, count_arr = text2vec_pipeline(text)        # something like [38761 10414 87130  6197], [1 1 1 1]
    w_td = (1 + np.log10(count_arr)) * idf_arr[id_arr] # there are several ways to compute w_td, here I follow the equation in the lecture
    c = 0                                              # for "how are you?", w_td = (1+np.log([1 1 1 1]))*idf_arr[[38761 10414 87130  6197]]
    for i in id_arr:                                   # for each token, inverted_file_dic[token][text_id] = w_td[c], c = 0~len(w_td)-1
        inverted_file_dic[id_token_dic[i]][text_id] = w_td[c]    
        c += 1                                         

In [85]:
for text_idx in q2_ser.index:                                     # for all the questions in question2
    add_text_to_inv_file_pipeline(q2_ser[text_idx], text_idx)     # add their information to inverted_file_dic

In [86]:
# this function will print top 5 matches and return a list of the answers
test = "What can one do after MBBS?"    

def ans_list(question, print_or_not = True, print_scores = False):           
    question_tokens = set(preprocessing_pipeline(question)) # pre-process the query
    ans_arr = np.zeros(init_data_len)                       # init_data_len = 363192, it is the initial length of Data
    for token in question_tokens:                           # for all the tokens in query
        temp_array = np.zeros(init_data_len)                # if {'what':{1:0.132, 5:0.249, 6:0.468...}}
        token_keys = np.array(list(inverted_file_dic[token].keys()), dtype = int)                 
        token_values = np.array(list(inverted_file_dic[token].values()))
        temp_array[token_keys] = token_values               # then inverted_file_dic['what'].keys() will be all the documents that 'what' appears
        ans_arr += temp_array                               # temp_array should be like [0, 0.132, 0, 0, 0, 0.249, 0.468...]
        
    ans_list = []
    for i in range(1,11):                                   # store 10 best matches in case some of the matches are the same as the query
        ans_list.append(np.argsort(ans_arr)[-i])
        if print_scores == True:
            print('top 10 scores:', np.argsort(ans_arr)[-i], ans_arr[np.argsort(ans_arr)[-i]])

    if print_or_not == True:
        print('the query is:', question)
    
    ans_list_top_5 = []
    count = 0
    for i in ans_list:
        if q2_ser[i] == question:                           # if a match is the same as the query
            pass                                            # do not do anything
        else:                                               # else
            if print_or_not == True:
                print(q2_ser[i])                            # print this match and add it to the list
            ans_list_top_5.append(q2_ser[i])
            count += 1
            if count == 5:
                break
    return ans_list_top_5

ans_list(test)

the query is: What can one do after MBBS?
Is it true that in order to get admission to usa after mbbs in india, ur medical college should be aiims only or u enter only on the basis of usmle? Basically does being from a good medical college in india guarantee you admission in usa or canada or some other country after mbbs or does it help?
How can I transfer from my MBBS medical college to another in Maharashtra after the 1st year of the MBBS?
I am first year MBBS student(ST) in one of new Aiims.I feel inferior to others due to my rank in entrance exam.can I get good results in MBBS exams?
What pay can one get after finishing mbbs in India on working abroad?
What are the possible career options in the USA after completing MBBS in India? If nothing, then what are the subjects that I can study after clearing the GRE and start a career there?


['Is it true that in order to get admission to usa after mbbs in india, ur medical college should be aiims only or u enter only on the basis of usmle? Basically does being from a good medical college in india guarantee you admission in usa or canada or some other country after mbbs or does it help?',
 'How can I transfer from my MBBS medical college to another in Maharashtra after the 1st year of the MBBS?',
 'I am first year MBBS student(ST) in one of new Aiims.I feel inferior to others due to my rank in entrance exam.can I get good results in MBBS exams?',
 'What pay can one get after finishing mbbs in India on working abroad?',
 'What are the possible career options in the USA after completing MBBS in India? If nothing, then what are the subjects that I can study after clearing the GRE and start a career there?']

In [87]:
def is_in_top(question, print_or_not = False):
    top_5_count = 0
    top_2_count = 0
    top_5_list = ans_list(question, print_or_not = print_or_not)
    top_2_list = top_5_list[0:2]
    top_5_set = set(top_5_list)
    top_2_set = set(top_2_list)
    if golden_ans_dict[question] in top_5_set:             # if the match is in top 5
        top_5_count += 1                                   # top_5_count = 1
        if(print_or_not == True):
            print("the answer is:", golden_ans_dict[question], '\n')
        if golden_ans_dict[question] in top_2_set:         # if the match is in top 5
            top_2_count += 1                               # top_2_count = 1
    elif print_or_not == True:                             # if the ground truth is not in top 5, then print 'not in top 5' and the answer
        print("not in top 5, the answer is:", golden_ans_dict[question],'\n')
    return top_5_count, top_2_count

In [88]:
top_5_count = 0
top_2_count = 0
for query in Q_100['question1']:
    top_5, top_2 = is_in_top(query, print_or_not = True)   # set print_or_not == False to cancel printing
    top_5_count += top_5
    top_2_count += top_2
print('top 5 count is: %d, probability is: %.2f' %(top_5_count,top_5_count/100))
print('top 2 count is: %d, probability is: %.2f' %(top_2_count,top_2_count/100))

the query is: How do I control my horny emotions?
Female Sexuality: How can I quell my horny nature and learn restraint so that I'm not disrespected?
I am a 22 years male. How do I become less horny?
How do I stop being so horny all the damn time?
How do I stop feeling so horny without masturbating all the time?
How do I make a woman horny?
not in top 5, the answer is: How do you control your horniness? 

the query is: What can one do after MBBS?
Is it true that in order to get admission to usa after mbbs in india, ur medical college should be aiims only or u enter only on the basis of usmle? Basically does being from a good medical college in india guarantee you admission in usa or canada or some other country after mbbs or does it help?
How can I transfer from my MBBS medical college to another in Maharashtra after the 1st year of the MBBS?
I am first year MBBS student(ST) in one of new Aiims.I feel inferior to others due to my rank in entrance exam.can I get good results in MBBS exa

## 4.  Sentence Embedding (Average)

In [89]:
# this is based on torchtext 0.12.0, torchtext.vocab
# if glove.6B.50d.txt is in the path, it will read the file in
# otherwise it will download
vec_dim = 50
vec = GloVe(name='6B', dim=vec_dim)

In [90]:
# to check a specific word vector
print(vec['how'])

tensor([ 6.8938e-01, -1.0644e-01,  1.7083e-01, -3.7583e-01,  7.5170e-01,
         7.8149e-04, -5.3102e-01, -1.9903e-01, -1.4419e-01,  1.2748e-01,
        -2.8038e-01,  7.0723e-01, -5.4100e-01,  1.9625e-01,  9.6635e-01,
         6.0519e-01,  4.0918e-01, -3.1612e-02,  5.3900e-01, -8.7086e-01,
        -2.0912e-01,  5.6853e-01,  6.5983e-01,  1.4583e-01,  1.0112e+00,
        -2.0736e+00, -1.1242e+00,  5.9662e-04,  7.0332e-01, -8.2608e-01,
         3.4445e+00,  3.2984e-01, -3.5324e-01, -1.0335e+00, -1.4753e-01,
        -1.4874e-01, -4.1246e-01,  3.3489e-01,  1.9841e-01, -2.5478e-01,
        -4.7193e-01,  6.6701e-02,  3.2777e-01,  6.8781e-01,  3.6428e-01,
         2.1522e-01,  1.6494e-01,  4.1761e-01, -2.2504e-01,  6.1412e-01])


In [91]:
# just for experiment
# nltk_sw = set(stopwords.words('english'))

In [92]:
# a function to calculate text vector
def wordvec2textvec_ave(mat):             # take a n*50 matrix composed of the word vectors of a text as parameter
    text_vec = torch.sum(mat, axis = 0)   # add up the vectors in mat
    text_vec = text_vec/len(mat)          # and calculate text vector by dividing the number of the tokens
    return text_vec

# a pipeline to convert text to a text vector
def text2vec_pipeline(text):        
    tokens = list(set(preprocessing_pipeline(text)))    # to remove stopwords, we can -nltk_sw from the set
    if len(tokens)!=0:
        mat = vec.get_vecs_by_tokens(tokens, lower_case_backup=True)
        res = wordvec2textvec_ave(mat)
        return res
    else:
        return torch.zeros(vec_dim)

test = "how are you?"
print("text vector of 'how are you?' is:\n", text2vec_pipeline(test))

text vector of 'how are you?' is:
 tensor([ 0.3761,  0.1860,  0.1983, -0.3616,  0.5707, -0.0812, -0.4278, -0.2987,
        -0.3166,  0.1438, -0.3641,  0.6457, -0.2467,  0.0857,  0.9632,  0.5897,
         0.4871,  0.1289,  0.2492, -0.9886, -0.3914,  0.3042,  0.7770,  0.4289,
         0.5693, -1.8318, -1.0873,  0.1897,  0.9493, -1.0442,  3.5759,  0.5769,
        -0.4228, -0.4108, -0.0913, -0.0152, -0.0469,  0.0914,  0.2712, -0.3119,
        -0.2951,  0.0863,  0.2563,  0.8156,  0.2807,  0.1937, -0.0559, -0.0784,
        -0.1370,  0.6317])


In [93]:
# this cell is only for showing my initial idea of calculating cos similarity,
# however, it was so slow that I had to switch to another method. Please see the report for details
def cos_sim(vec1, vec2):                                                                           # cos_sim = a·b/(||a||·||b||)
    if float(torch.sqrt(torch.sum(vec1**2))) == 0 or float(torch.sqrt(torch.sum(vec2**2))) == 0:   # if ||a|| or ||b|| is 0
        return float(0)                                                                            # return 0
    res = torch.inner(vec1,vec2)/(torch.sqrt(torch.sum(vec1**2)) * torch.sqrt(torch.sum(vec2**2))) # else, return a·b/(||a||·||b||)
    return res

In [94]:
# a function to calculate ||vec||
def vec_length(vec):
    return torch.sqrt(torch.sum(vec**2))

In [95]:
q2_mat = np.zeros([init_data_len, vec_dim])       # initialize a matrix for question2
q2_vec_len = np.zeros(init_data_len)              # initialize an array for the ||vec|| of the vectors in the matrix
for i in q2_ser.index:                            # for all the questions in question 2
    vec_temp = text2vec_pipeline(q2_ser[i])       # convert it to a vector
    q2_mat[i] = vec_temp                          # and add it to the matrix
    q2_vec_len[i] = vec_length(vec_temp)          # add ||vec|| of it to the array
    
q2_mat_T = q2_mat.T
q2_vec_len = q2_vec_len.reshape(1,init_data_len)
print("the shape of q2_mat_T is:",q2_mat_T.shape)
print("the shape of q2_vec_len is:",q2_vec_len.shape)

the shape of q2_mat_T is: (50, 363192)
the shape of q2_vec_len is: (1, 363192)


In [96]:
# this function will print top 5 matches and return a list of the answers
test = "What can one do after MBBS?"

def ans_list(question, print_or_not = True, print_scores = False): 
    query_vec = text2vec_pipeline(question)             # convert query to a text vector
    query_len = vec_length(query_vec)                   # calculate its ||vec||
    inner_mat = np.matmul(query_vec,q2_mat_T)           # multiply the query vector and the quesiont2 matrix
    normalize_fac = np.array(query_len * q2_vec_len)    # an array of ||a||·||b||
    q2_score_arr = np.divide(inner_mat, normalize_fac, out=np.zeros_like(normalize_fac), where=normalize_fac!=0)
    q2_score_arr = q2_score_arr.flatten()               # a·b/(||a||·||b||)
        
    ans_list = []
    for i in range(1,11):                               # store 10 best matches in case some of the matches are the same as the query
        ans_list.append(np.argsort(q2_score_arr)[-i])
        if print_scores == True:
            print('top 10 scores:', np.argsort(q2_score_arr)[-i], ans_arr[np.argsort(q2_score_arr)[-i]])

    if print_or_not == True:
        print('the query is:', question)
        
    ans_list_top_5 = []
    count = 0
    for i in ans_list:
        if q2_ser[i] == question:                       # if a match is the same as the query
            pass                                        # do not do anything
        else:                                           # else
            if print_or_not == True:
                print(q2_ser[i])                        # print this match and add it to the list
            ans_list_top_5.append(q2_ser[i])
            count += 1
            if count == 5:
                break
    return ans_list_top_5    

ans_list(test)

the query is: What can one do after MBBS?
What have to Do after MBBS?
What can we do after completing BDs?
What can I do after my MBBs?
Can I do a PhD after a CA?
What should I do and not do before an exam?


['What have to Do after MBBS?',
 'What can we do after completing BDs?',
 'What can I do after my MBBs?',
 'Can I do a PhD after a CA?',
 'What should I do and not do before an exam?']

In [97]:
# this cell is the same as previous
def is_in_top(question, print_or_not = False):
    top_5_count = 0
    top_2_count = 0
    top_5_list = ans_list(question, print_or_not = print_or_not)
    top_2_list = top_5_list[0:2]
    top_5_set = set(top_5_list)
    top_2_set = set(top_2_list)
    if golden_ans_dict[question] in top_5_set:
        top_5_count += 1
        if(print_or_not == True):
            print("the answer is:", golden_ans_dict[question], '\n')
        if golden_ans_dict[question] in top_2_set:
            top_2_count += 1
    elif print_or_not == True:
        print("not in top 5, the answer is:", golden_ans_dict[question],'\n')
    return top_5_count, top_2_count

In [98]:
top_5_count = 0
top_2_count = 0
for query in Q_100['question1']:
    top_5, top_2 = is_in_top(query, print_or_not = True)
    top_5_count += top_5
    top_2_count += top_2
print('top 5 count is: %d, probability is: %.2f' %(top_5_count,top_5_count/100))
print('top 2 count is: %d, probability is: %.2f' %(top_2_count,top_2_count/100))

the query is: How do I control my horny emotions?
How can I control my emotions?
How do I regulate my emotions?
How do I control my feelings of liking someone?
How should I control my emotion?
How do I control my temper?
not in top 5, the answer is: How do you control your horniness? 

the query is: What can one do after MBBS?
What have to Do after MBBS?
What can we do after completing BDs?
What can I do after my MBBs?
Can I do a PhD after a CA?
What should I do and not do before an exam?
not in top 5, the answer is: What do i do after my MBBS ? 

the query is: What is the best self help book you have read? Why? How did it change your life?
Do you have a book that changed your life?
What is the one book that you think reading it will change my life to the better immediately?
What is the best life advice you would give?
Did you ever witness something that made you change your perspective on life?
What can I learn on the internet that can change my life?
not in top 5, the answer is: What

## 5.  Sentence Embedding (Simple But Tough-to-beat)

In [99]:
# this cell is to calculate p(w) in the formula
unigram_freq = collections.Counter(q_tokens_stm)          # q_tokens_stm is all tokens of all the questions
q_tokens_stm_len = len(q_tokens_stm)                      # the length of q_tokens_stm
p_dict = {}                                          

for token in set(q_tokens_stm):                           # for each token 
    p_dict[token] = unigram_freq[token]/q_tokens_stm_len  # add its p(w) to the dictionary
    
def p_arr_fn(token_list):                                 # this function is to form p(w) array for a specific token list
    p_list = []
    for token in token_list:                              # for each token in token list
        p_list.append(p_dict[token])                      # add its p(w) to the p(w) list
    p_arr = np.array(p_list)                              # convert the p(w) list to an array
    return p_arr    

In [100]:
def wordvec2textvec_simple(token_list):
    s = len(token_list)     # the length of the token list of a sentence
    temp_list = []
    a = 0.0001              # a is given by the paper
    weighted_sum = 0
    mat = vec.get_vecs_by_tokens(token_list, lower_case_backup=True)     # form a matrix of all the word vectors
    mat_T = np.array(mat.T)                                              # if a text has 4 tokens, then mat_T should be 50×4
    p_arr = p_arr_fn(token_list)                                         # p_arr should be 1×4
    weighted_sum = np.sum((a*mat_T) / (a+p_arr), axis = 1)               # this is the formula of the first step
    sen_vet = weighted_sum/s                                             # divide the weighted sum by the length
    return sen_vet

# a pipeline to convert text to a text vector
def text2vec_pipeline(text):        
    tokens = list(set(preprocessing_pipeline(text)))                     # to remove stopwords, we can -nltk_sw from the set
    if len(tokens)!=0:
        res = wordvec2textvec_simple(tokens)
        return res
    else:
        return np.zeros(vec_dim)

test = "how are you?"
print("text vector of 'how are you?' is:\n", text2vec_pipeline(test))

text vector of 'how are you?' is:
 [ 2.53341868e-03  1.01487294e-03  1.66176003e-03 -2.31766862e-03
  3.96812984e-03 -9.00258033e-04 -2.55204233e-03 -1.43911254e-03
 -1.83625803e-03  9.61233199e-04 -2.21647019e-03  4.35315762e-03
 -1.06674382e-03  6.37576893e-04  5.74732227e-03  3.95404635e-03
  3.28858804e-03  8.53517297e-04  1.83186762e-03 -7.36398322e-03
 -2.26972420e-03  1.92667625e-03  5.12336021e-03  3.18157921e-03
  4.08854062e-03 -1.13141713e-02 -6.64651003e-03  1.35467387e-03
  6.01234651e-03 -7.48911954e-03  2.38040875e-02  4.18807231e-03
 -2.47782003e-03 -2.27910594e-03 -4.37990828e-04  1.82794317e-04
 -2.73679991e-05  6.25717487e-04  1.56789116e-03 -2.28253196e-03
 -1.21654040e-03  4.91722112e-04  1.30855580e-03  5.38163818e-03
  1.90158206e-03  1.12116379e-03  2.65113468e-06 -6.81277230e-04
 -1.07682804e-03  3.76562442e-03]


In [101]:
# reset the index of q1_q2_concat for later use
q1_q2_concat = q1_q2_concat.reset_index(drop=True)
q1_q2_concat

0         How is the life of a math student? Could you d...
1                       How do I control my horny emotions?
2              What causes stool color to change to yellow?
3                               What can one do after MBBS?
4         Where can I find a power outlet for my laptop ...
                                ...                        
493129                  How can I get the best out of life?
493130      What is the demand of Oracle ATG in the market?
493131                                  How thick is 5 mil?
493132       How does a generator work and produce current?
493133    How do I understand the difference between the...
Length: 493134, dtype: object

In [102]:
X_mat = np.zeros([len(q1_q2_concat), vec_dim])           # to form a matrix of text vectors
for i in range(len(q1_q2_concat)):                    
    X_mat[i] = text2vec_pipeline(q1_q2_concat[i])        # each row is a 50-dimension vector. Suppose we have 40,000 text, X_mat should be 40000×50

X_mat_T = X_mat.T                                        # X_mat_T should be 50×40000
print("the shape of X_mat_T is:", X_mat_T.shape)

the shape of X_mat_T is: (50, 493134)


In [103]:
U, S, Vh = np.linalg.svd(X_mat_T, full_matrices=False)   # implement SVD

In [104]:
# take the first left singular vector u, first right singular vector v, and the first singular value σ
# for more details please see the report
first_U_column = U.T[0]    # u
first_Vh_row = Vh[0]       # v
first_sig_value = S[0]     # σ

In [105]:
# X = X - σuv
X_mat_T_modified = X_mat_T - (first_sig_value * np.outer(first_U_column, first_Vh_row))
X_mat_modified = X_mat_T_modified.T
print("the shape of X_mat_modified is:", X_mat_modified.shape)

the shape of X_mat_modified is: (493134, 50)


In [106]:
def vec_length(vec):
    return np.sqrt(np.sum(vec**2))

In [107]:
X_mat_modified_len = np.zeros(len(X_mat_modified))             # ||vec|| for all the text vectors
for i in range(len(X_mat_modified)):                           # for each vector in X_mat_modified
    X_mat_modified_len[i] = vec_length(X_mat_modified[i])      # calculate its ||vec|| and add it to the length array
    
X_mat_modified_len = X_mat_modified_len.reshape(1,len(X_mat_modified))
print("the shape of X_mat_modified_len is:", X_mat_modified_len.shape)

the shape of X_mat_modified_len is: (1, 493134)


In [108]:
question_dict = {}                                # a dictionary like {q1:vec1, q2:vec2, q3:vec3...}
for i in range(len(q1_q2_concat)):
    question_dict[q1_q2_concat[i]] = X_mat_modified[i]

In [109]:
q2_mat = np.zeros([init_data_len, vec_dim])       # form a matrix for question2 column
q2_vec_len = np.zeros(init_data_len)              # and an array for each ||vec|| (just like averaging method)
for i in q2_ser.index:
    vec_temp = question_dict[q2_ser[i]]           # q2_ser[i] is a specific question, like 'how are you?'. Find its vector from the question_dict
    q2_mat[i] = vec_temp                          # update q2_mat defined above
    q2_vec_len[i] = vec_length(vec_temp)          # then calculate its ||vec|| and add it to the array
    
q2_mat_T = q2_mat.T
q2_vec_len = q2_vec_len.reshape(1,init_data_len)
print("the shape of q2_mat_T is:", q2_mat_T.shape)
print("the shape of q2_vec_len is:", q2_vec_len.shape)

the shape of q2_mat_T is: (50, 363192)
the shape of q2_vec_len is: (1, 363192)


In [110]:
# this function will print top 5 matches and return a list of the answers
# this step is the same as the averaging method
test = "What can one do after MBBS?"

def ans_list(question, print_or_not = True, print_scores = False):
    query_vec = question_dict[question]
    query_len = vec_length(query_vec)
    inner_mat = np.matmul(query_vec,q2_mat_T)
    normalize_fac = np.array(query_len * q2_vec_len)    
    q2_score_arr = np.divide(inner_mat, normalize_fac, out=np.zeros_like(normalize_fac), where=normalize_fac!=0)
    q2_score_arr = q2_score_arr.flatten()
        
    ans_list = []
    for i in range(1,11):                              # store 10 best matches in case some of the matches are the same as the query
        ans_list.append(np.argsort(q2_score_arr)[-i])
        if print_scores == True:
            print('top 10 scores:', np.argsort(q2_score_arr)[-i], ans_arr[np.argsort(q2_score_arr)[-i]])

    if print_or_not == True:
        print('the query is:', question)
        
    ans_list_top_5 = []
    count = 0
    for i in ans_list:
        if q2_ser[i] == question:                      # if a match is is the same as the query
            pass                                       # do not do anything
        else:                                          # else
            if print_or_not == True:
                print(q2_ser[i])                       # print this match and add it to the list
            ans_list_top_5.append(q2_ser[i])
            count += 1
            if count == 5:
                break
    return ans_list_top_5    

ans_list(test)

the query is: What can one do after MBBS?
What have to Do after MBBS?
What do i do after my MBBS ?
What can I do after my MBBs?
How should I study in MBBS?
How should I study first year MBBS?


['What have to Do after MBBS?',
 'What do i do after my MBBS ?',
 'What can I do after my MBBs?',
 'How should I study in MBBS?',
 'How should I study first year MBBS?']

In [111]:
# this cell is the same as previous
def is_in_top(question, print_or_not = False):
    top_5_count = 0
    top_2_count = 0
    top_5_list = ans_list(question, print_or_not = print_or_not)
    top_2_list = top_5_list[0:2]
    top_5_set = set(top_5_list)
    top_2_set = set(top_2_list)
    if golden_ans_dict[question] in top_5_set:
        top_5_count += 1
        if(print_or_not == True):        
            print("the answer is:", golden_ans_dict[question], '\n')
        if golden_ans_dict[question] in top_2_set:
            top_2_count += 1
    elif print_or_not == True:
        print("not in top 5, the answer is:", golden_ans_dict[question],'\n')
    return top_5_count, top_2_count

In [112]:
top_5_count = 0
top_2_count = 0
for query in Q_100['question1']:
    top_5, top_2 = is_in_top(query, print_or_not = True)
    top_5_count += top_5
    top_2_count += top_2
print('top 5 count is: %d, probability is: %.2f' %(top_5_count,top_5_count/100))
print('top 2 count is: %d, probability is: %.2f' %(top_2_count,top_2_count/100))

the query is: How do I control my horny emotions?
How do I stop feeling so horny without masturbating all the time?
What do girls do when they feel horny?
How can I not be so horny?
Why am I horny?
Why don't I get horny?
not in top 5, the answer is: How do you control your horniness? 

the query is: What can one do after MBBS?
What have to Do after MBBS?
What do i do after my MBBS ?
What can I do after my MBBs?
How should I study in MBBS?
How should I study first year MBBS?
the answer is: What do i do after my MBBS ? 

the query is: What is the best self help book you have read? Why? How did it change your life?
Which is the best self help book you've ever read?
Which books should I read now in self improvment?
What are the top self help books I should read?
What is the answer to tell me about your self interview?
Which books can help us for self study?
the answer is: What are the top self help books I should read? 

the query is: What will be Hillary Clinton's policy towards India if 

***
CHEN Chen a1819099, 16-Apr-2022